# Introduction
 

 Taking derivatives is an ubiquitous task that covers a wide range of applications - from mathematics, to physics and computer science, etc. 
 
 In machine learning, differentiation is extensively applied to the optimization problem by performing the gradient descent to minimize the cost function. 
Using a fast and versatile differentiation method, people can also get the second order derivatives and hessian matrices.
 
 Differentiation is also applied to solve everyday problems. For example, knowing the initial decomposition level of a contaminated food, 
 we can define a differential equation to predict how food will decay in the future and set an appropriate expiration date. 
 
 However, traditional methods of differentiation such as symbolic differentiation and numerical differentiation might be computationally expensive and less robust. Also, these methods do not scale well to vector functions with multiple variable inputs, which are widely used to solve real world problems.

 In this package, SmartDiff is a software package which implements a forward mode Automatic Differentiation (AD) package with chain rule to help users take derivatives easily. It automatically breaks the complex functions into small independent elementary operations based on parentheses and operators. Then it calculates the derivatives in the small chunks step by step, moving outward to the entire function, to give the result. Therefore, SmartDiff is widely **generalizable** to different functions and **consumes very little memory**. For vector-valued functions, it computes the derivative of one component at a time.

 Calculating higher order derivatives can be very useful for optimization problems. For example, the first order derivative can only gets us to the stationary points. Without the second order derivative information, we cannot tell whether a stationary point is a local extremum. Therefore, we choose to implement the higher order derivatives as the extension.


# Background

## Chain Rule
 This automatic differentiation (AD) is performed mainly based on the chain rule. 
 The basic idea is to split the function into hierarchies of chunks containing the input variables and the built-in elementary operations in SmartDiff. 
 Therefore, from the innermost chunk(s), the calculation can be expressed in a series of steps iteratively, in which SmartDiff only considers one elementary operation at a time. 
 After the evaluation of both the function and its derivatives, the iteration can be traced in the computation graph. 

 Based on the key definition of chain rule, and known that $F(x) = f(g(x))$, we can represent it in multiple steps as follows:
    $\frac{d}{d x}(F(x)) =\frac{d}{d x}f(g(x))=f'(g(x))g'(x)$

 From the equation above, as long as we can compute the derivative of $f(x)$ and $g(x)$, we can get the derivative of $F(x)$.

Also, with such forward method, SmartDiff is able to evaluate each step of the calculation and keep track of function values and derivatives at each step, which makes the calculation faster and more robust.

We note that the chain rule also applies to the multivariate input to calculate the Jacobian matrix (1st order derivative). 
More specifically, the entry of the Jacobian matrix at the $i$th row and the $j$th column is the derivative of 
the $i$th component in the input function with respect to $x_j$. This can be calculated by setting only $x_j$ to be an
AutoDiff object and $x_i, i \ne j$ to be scalars.

## Recursive Chain Rule for Calculating the higher order derivatives
In order to get nth order derivative of functions, we start by writing out the nth order derivative for each 
elementary functions and operations. For example, we can write up the nth order derivative of sine function 
because there is an explicit formula for it. e.g. 
$$
\frac{d^n}{d x^n} \sin(x) = \sin(x+\frac{\pi n}{2})
$$
Alternatively, when this formula doesn't exist for a function, we express the function as a composite function 
of other elementary operations and functions. e.g. the hyperbolic tangent function is a composite function from the exponential function
$$
\tanh(x) = \frac{e^{2x}-1}{e^{2x}+1}=f(g(y)) \text{ where $f(x)=\frac{x-1}{x+1}=1-\frac{2}{x+1}$ and $g(x)=e^{2x}$} 
$$
Since we know the n-th order derivative of $e^x$ and the division operator, we can calculate the derivative of the
composite function using the Faa di Bruno's formula：

$$
\frac{d^n}{d x^n} f(g(x)) = \sum_{k=1}^n f^{(k)}(g(x)) \cdot B_{n,k} (g'(x), g''(x),..., g^{n-k+1}(x))
$$

This formula involves a Bell polynomial ($B_{n,k}$) and derivative values from 1 to n-1th order of $f$ and $g$, 
and we have the formula to calculate these derivatives. Similarly, we implemented the Hessian calculation for real-valued functions with multivariable input.


## Graph Structure of Calculation
 SmartDiff is implemented based on a graph structure. For each node on the graph, it contains a variable or the result following one elementary operation. 
 For example, given a multivariable function $f(x,y) = (x^2+1)+\exp{(y)}$. To evaluate the function at $(x,y) = (1,1)$, we can write the evaluation trace as:

|     `Trace`        |   ` Operation`     |        `Value`         |    ` Derivative `       |  `Der to x `|            `Der to y`            |
|: -----------------:|:------------------:|:----------------------:|:-----------------------:|:-----------:|:--------------------------------:|
|        $x_1$       |       $x$          |          1             |       $ \dot{x}$        |       1     |                 0                |
|        $x_2$       |       $y$          |          1             |       $ \dot{y}$        |       0     |                 1                |
|        $v_1$       |     $x_1^2$        |          1             |       $2x_1\dot{x_1}$   |       2     |                 0                |
|        $v_2$       |     $v_1+1$        |          2             |          $\dot{v_1}$    |       2     |                 0                |
|        $v_3$       |     $\exp{(x_2)}$  |          $e$           |  $\exp{(x_2)}\dot{x_2}$ |       0     |               $e$                |
|         $f$        |     $v_2+v_3$      |          $e+2$         |  $\dot{v_2}+\dot{v_3}$  |       2     |               $e$                |

The traces above split the function into chunks of elementary operations and can be easily put into a graph structure as following, which is helpful for the differentiation calculation.

![graph_structure](figures/graph.PNG)

## Elementary Functions
 SmartDiff is mainly a combination of elementary operations in calculus, which contains:
 
 1. Arithmetic symbols (+, -, ×, /), parentheses, constant functions ($2$, $\pi$, $e$, etc.)
 2. Logarithmic functions, exponential functions, trigonometric functions, square root function, and logistic functions.
 3. Comparison operators of different AutoDiff objects, including $\lt$, $\gt$, $\leq$, $\geq$, =, $\ne$.

# How to use SmartDiff

**New screenshots**

1. Installation 

    - **From Github:** The user first installs SmartDiff and calls the main script in the commandline and interacts with the Graphic User Interface (GUI). **There is no need to import or instantiate python objects.**
    
        ```python
            git clone https://github.com/SmartDiff/cs107-FinalProject.git
            cd /dir/to/SmartDiff
            python main.py
        ```
        
        PyQt5 should be automatically installed as it is specified in requirements.txt. In case the user gets "cannot find module PyQt5" in the command line, do the following to install PyQt5 (default 5.15.1)

        ```python
            pip install PyQt5
        ```

    - **From PyPI**: SmartDiff is also available on PyPI https://pypi.org/project/Smartdiff/0.0.16/ and all the dependency should be automatically installed via the command below.

        ```python
            pip install Smartdiff==0.0.16
            cd /dir/to/SmartDiff
            python main.py
        ```

2. Interaction with GUI

    **Base feature**

    Calling the main.py script in the command line should initiate a GUI. Given SmartDiff can take an arbitrary order of derivatives, the user chooses the order of derivatives in step 0. For the base case, choose N=1 to get the Jacobian matrix.

    <img src="figures/step0.PNG" style="width:500px;">

    Based on the derivative order, SmartDiff can override user input and display a warning message in the main window (more in the next section). In the case of first order derivatives, there is no constraint on the dimension of the input variable or the function.

    <img src="figures/step1.PNG" style="width:500px;">

    User can choose the dimension of the vector function and the number of input variables. Click ok to continue. The instructions to input the variables and function components are listed here for later steps.

    Once the user clicks OK, the user sees the second step GUI, where the user can put in a float between -100 and 100 with up to 4 decimals for each variable value.

    <img src="figures/step2.PNG" style="width:500px;">

    Click OK or press Enter to continue to step 3 as shown below. Do not press Cancel!

    <img src="figures/step3.PNG" style="width:500px;">

    The user can input any function, one component at a time, which consists of elementary functions and operations defined in SmartDiff based on the instructions in the main window. Again, press OK or press Enter to continue to step 4. **Make sure to follow the x0, x1, x2, ... convention for input variables**. Note that SmartDiff has some built-in simple error handling, so $x$, $x0$, and $x1$ will later be recognized as $x0$.
 
    <img src="figures/step4.PNG" style="width:500px;">

    This is for the user to confirm the input (no change allowed at this stage). The user can choose to show the function evaluated at this point by clicking the checkbox. Click OK to continue to step 5.

    <img src="figures/step5.PNG" style="width:500px;">

    SmartDiff is able to handle the error message during differentiation and function evaluation. In this case, $\log_2(0)$ is an invalid operation and the error message is shown. The user is then instructed to go back to step 1 to fix the issue.

    Then, if the user evaluates the function at $x0=1$, SmartDiff runs successfully as shown below.

    <img src="figures/step4_fix.PNG" style="width:500px;">

    <img src="figures/step5_fix.PNG" style="width:500px;">

    Press Quit to exit the program or close step 4 and 5 windows to differentiate another function.

    **Extended feature**

    SmartDiff allows calculation of higher order derivatives. If the function is real-valued with univariate input, SmartDiff allows an arbitrary order of derivatives. 
    For example, we can take the 10th order derivative of $x_0^{12}$ evaluated at $x_0=12$.

    <img src="figures/step4_high.PNG" style="width:500px;">

    Similar to the base case, SmartDiff outputs the results and, if applicable, error messages during the computation.

    <img src="figures/step5_high.PNG" style="width:500px;">

    Another case SmartDiff supports is the Hessian matrix computation of a scalar-valued function with multivariate input.  
    For example, we can calculate the Hessian of function $x_0+x_1+x_2^2$ at $x_0=1, x_1=2, x_2=3$.

    <img src="figures/step4_hessian.PNG" style="width:500px;">

    <img src="figures/step5_hessian.PNG" style="width:500px;">

    However, SmartDiff does not handle other form of derivatives, such as derivatives higher than 2nd order for functions with multivariable input or derivatives higher than 1st order for vector-valued functions. If the user violates this restriction, SmartDiff overrides user specification and shows a warning message in the main window, so the user can take that into account when inputting the function and variable values.

    <img src="figures/step1_warning.PNG" style="width:500px;">

# SmartDiff Graphic User Interface (GUI)

### Procedure:

Once SmartDiff is installed, it outputs the derivative(s) in the following steps.  

   1. The user inputs the order of derivative 
   
   2. The user chooses dimension of the vector function from a drop down list (1,2,3) and the number of one-dimensional real-valued variables from a drop down list (1,2,3)
  
   2. The user inputs a specific data at point to evaluate the derivative (required, default 0)

   3. The user inputs a vector function by each component of the vector function (required)

   4. The user checks the confirmation message, showing the function and the variable values and chooses whether to see the function value. The user clicks ok to compute the derivative

   5. The program shows the value of the function (optional) and the value of the derivative

### Advantages:

GUI is more user friendly than command line interface. We have also implemented checking functions to make sure the user input does not contain unexpected characters or unclosed parentheses.
In addition, since SmartDiff currently don't support higher order ($\gt 1$) derivatives for vector functions and ($\gt 2$) derivatives for multivariate inputs, upon selection of the order of derivatives, SmartDiff overrides user selection and displays a warning message.

# Software Organization

This is a flow chart of the relationship of different modules in SmartDiff.

<img src="figures/Flowchart.png" style="width:500px;">

### 1. Directory structure:
The following is the structure of the project after users download it from our github repository. 
The package we are developing is SmartDiff and the few files outside of it would be used in its packaging and distribution.
```python
    cs107-FinalProject/
        .travis.yml # for TravisCI and Codecov
        LICENSE # for PyPI packaging
        MANIFEST.in # for non-python files to be packaged into PyPI distribution
        requirements.txt # for Github and PyPI installation
        README.md
        setup.py # for PyPI packaging
        
        docs/
            milestone1.ipynb
            milestone2.ipynb
            milestone2_progress.ipynb
            documentation.ipynb
            figures/ # illustrations of how to use
                Flowchart.PNG
                graph.PNG
                step0.PNG
                step1.PNG
                step1_warning.PNG
                step2.PNG
                step3.PNG
                step4.PNG
                step4_fix.PNG
                step4_hessian.PNG
                step4_high.PNG
                step5.PNG
                step5_fix.PNG
                step5_hessian.PNG
                step5_high.PNG
            
        SmartDiff/
            __init__.py
            globals.py # defines the global mapping functions for python eval parser
            main.py # contains the GUI python code (PyQt5) and calls modules in solvers and preprocess.
            preprocess/  # This is a (sub)package
                __init__.py 
                pyexpr_formatter.py  # A GUI input preprocessing module
                pyexpr_formatter_test.py  # test script for pyexpr_formatter.py
            tests/ # all the test suits
                high-order_test.py  # test script for higher order operations in element_op.py
                pyexpr_formatter_test.py  # test script for pyexpr_formatter.py
                solvers_test.py  # test script for element_op.py and multivariate.py
            solvers/ #  calculates derivatives and function values at the given point
                __init__.py
                element_op.py  # defines the derivatives of elementary operations and go through the preprocessed python expression and call the corresponding elementary operations
                multivariate.py  # defines the Jacobian and Hessian calculations
            GUI/  # stores .ui files drawn in Qt Designer
                step1.ui  # main window
                step4.ui  # shows user input
                step5.ui  # shows SmartDiff output                
```

### 2. Testing Procedure
We started by implementing some basic test cases in the docstring. For extensive testing between the boundaries of different modules, we will implement test functions for each module
on separate files within the same directory. The tests we have implemented so far cover all functions from *element_op.py*, *multivariate.py*, *pyexpr_formatter.py*, which check the correct definition and
calculation of the derivatives for elementary operations, handling multivariate input and vector functions, and user input string formatting, respectively.

The automatic testing of our code is conducted using TravisCI and the proportion of code tested is monitored by CodeCov.

### 3. Packaging and Package Distribution
See How to Use: Installation. The developers and the consumers should follow the same procedure for installation.

### 4. Basic idea of SmartDiff:

From the user input in GUI, we can simultaneously

1. get a python expression from user input string

2. constrain user input dimensions (variable and function) to a limited set, based on the derivative order

3. display input instructions, results, and error messsages

For each component of the vector function, we start from the inner most expression and propagate outward as in forward mode of automatic differentiation.

# Implementation

The overall idea of our SmartDiff is to first format the user's input expression into a python expression. And then use Python's built-in eval() function to evaluate the result.
Since Python compiler needs to know the value of each variable in the expression in runtime, we have our GUI to register the input variable name as a string with its corresponding value.
Next, we use the forward mode AD to create a seed AD object from the target differentiation variable, leaving others as variable mapped to the user-defined values. 
Finally, we use Python's eval() function on the formatted string expression and the hashmap of variable to the corresponding AD object to evaluate the final function value and derivative.   


### 1. Core Data Structures
The core data structures are queue, stack and hashmap. The queue is used to store the filtered characters when we are formatting the input via linear scanning.
It is also used to evaluate the mathematical expression in Python. Stack is used by Python internally to evaluate expressions nested in parenthesis. 
Hashmap is used by our GUI to store the input variable name and value. We use numpy array for the automatic differentiation module and the GUI.

### 2. Classes, Methods and Name Attributes

#### PyExpression_Formatter() class  
This class formats a user input to what our project could handle. For example, it checks whether the input string is a valid function expression and can fix certain ill expressions.
   - \[Field\] special_char_set: A set of legal special characters in the user's function input   
   - \[Method\] is_valid_input: 0 if input string is valid, 1 if input string contains mismatched parenthesis, 2 if input string contains illegal characters
   - \[Method\] format_to_pyexpr: Formats the input string to a Python compatible and compilable math expression 
    

#### AutoDiff() class
The fundamental class in the backend computation is this AutoDiff object. SmartDiff creates an AD object after each step in the computation. 
Therefore, each AD object can be thought of as a final representation of a mathematical term whose val is the numerical value up to that point of computation and
der is the derivative with respect to certain target derivative variable.  


##### Primitive operators
To handle basic operations, we use operator overloading where the following primitive operators are overwritten in the way such that each operator outputs an AD object
with the numerical value as well as the derivative w.r.t the input variable stored in its fields.
   - \[Field\] val: Numerical value of the current chunk of expression
   - \[Field\] der: A length-N vector where der\[i\] is the i^th derivative of the AD term with respect to the input variable  
   - \[Field\] N: Differentiation order (default to 1)
   - \[Method\] \__add\__(other): Adds the value of two AutoDiff expression and their derivative
   - \[Method\] \__radd\__(other): Calls \__add(other)\__ when the left operand is not an AutoDiff object
   - \[Method\] \__sub\__(other): Subtracts the value of two AutoDiff expression and their derivative
   - \[Method\] \__rsub\__(other): Calls \__sub(other)\__ when the left operand is not an AutoDiff object
   - \[Method\] \__mul\__(other): Multiplies the value of two AutoDiff expression and computes the final derivative
   - \[Method\] \__rmul\__(other): Calls \__mul(other)\__ when the left operand is not an AutoDiff object
   - \[Method\] \__truediv\__(other): Divides the value of two AutoDiff expression and computes the final derivative
   - \[Method\] \__rtruediv\__(other): Calls \__truediv(other)\__ when the left operand is not an AutoDiff object
   - \[Method\] \__pow\__(other): Powers the value of two AutoDiff expression and their derivative
   - \[Method\] \__rpow\__(other): Calls \__pow(other)\__ when the left operand is not an AutoDiff object
   - \[Method\] \__neg\__(): Negates the value and derivative of the current AD object
   - \[Method\] \__pos\__(): A unary plus operator that does not change the value or derivative of the AD object

##### Comparison operators
Moreover, we also support a series of comparison operators to compare two AD objects as follows. For now, we define the comparison field to val because
it is the most straightforward and intuitive comparison field. In the future, depending on the use case (e.g. compare derivative in gradient descent), 
users can modify the following methods such that they compare the der field.
   - \[Method\] \__lt\__(other): True if self is less than other in val
   - \[Method\] \__gt\__(other): True if self is greater than other in val
   - \[Method\] \__le\__(other): True if self is less than or equal to other in val
   - \[Method\] \__ge\__(other): True if self is greater than or equal to other in val
   - \[Method\] \__eq\__(other): True if self is equal to other in val
   - \[Method\] \__ne\__(other): True if self is not equal to other in val

##### Elementary math operators
Sometimes, the user wants to input a function with some elementary math functions, such as log, sin, tanh, etc. 
In order to enable our AD computation along evaluating these functions, we override these math operations with our own implementation that
deals specifically with AD objects. Each function we implement below handles all bases when the input x is or is not an AD object. We use the basic
calculus rule specific to each math function to accumulate the computation of the derivative. Each returns an AD object whose value and derivative are
computed and assigned in that function accordingly.
   - power(x,n)
   - log(x,n)
   - ln(x)
   - expn(x,n)
   - exp(x)
   - inv(x)
   - sqrt(x)
   - sin(x)
   - cos(x)
   - tan(x)
   - arcsin(x)
   - arccos(x)
   - arctan(x)
   - sinh(x)
   - cosh(x)
   - tanh(x)
   - logistic(x)
    
    
GUI() related classes  
This class will be based on the python GUI package, PyQt5 and inherits from the following predefined python classes to 
inform the user with instructions and error messages, direct the user to input the information, and display the output:
   - QMainWindow, QDialog, QInputDialog (different windows and dialogs)
   - QComboBox, QLineEdit, QPushButton, QLabel, QCheckBox (different components in the windows/dialogs)

### 3. External Dependencies
#### Installation dependencies
   - PyQt5 (5.15.1), for the GUI
   - NumPy, for the solvers
   - SymPy, for computing the Bell polynomial in higher order derivatives, function evaluation, and Binomial coefficients
   - Pydoc, for viewing the function docstring
   - doctest, for unit testing of all the cases a function deals with (in a function docstring)
   - pytest, for testing files in test (extensive testing of boundaries between different modules)

#### PyPI distribution dependencies
The following are not required to install SmartDiff, but are essential to building and packaging our application:
   - setuptools & wheel, for packaging and distributing the code
   - twine, for uploading the distribution archive files


# Broader Impact and Inclusivity Statement 

Although great improvements have been made in the world of STEM with regards to diversity and inclusivity, 
it is no secret that a lot of work remains to be done to bridge the gap between technological advancements 
and underrepresented groups. As we designed this software, we brainstormed ways to make it as inclusive as 
possible, recognizing that people from diverse technical backgrounds might engage with it in the future. 
Therefore, we decided to implement a graphic user interface (GUI) to make the user experience as friendly 
and smoothly as possible. That being said, although interacting with the software is straightforward, the 
interface is designed for English speakers and assumes familiarity with elementary and multivariable functions,
which could still act as barriers to future users. More work needs to be done to mitigate these effects as 
much as possible, and to make the user experience as informative and clear as possible. 

Our software has both positive and negative implications, but here we will argue that the positive ones 
outnumber the negative ones. On a technical level, we worked on making this software as time- and memory-
efficient as possible while also supporting complex operations involving higher order derivatives and 
multivariable functions. Since efficiency and scalability are important components of automatic differentiation, 
we believe we are contributing a new software to the broader community, making it more diverse. Also, as 
mentioned before, we directed our efforts to software inclusivity, though we recognize that more work in 
this area needs to be done. Finally, we are aware that job loss is a main concern that comes with several 
types of automation. However, instead of disrupting professional opportunities, we sincerely hope our tool 
will instead create opportunities for more people interested in using AD in their areas of research and work
and are currently not doing due to a lack of user-friendly software. However, in the case of such disruption
ever existing, efforts should be made to instruct people at risk on the usage of this software while making
the transition as easy as possible.

# Our extension

We would like to implement taking second derivatives of a given input function, i.e. the Hessian matrix. 
Then using the Hessian matrix and the Jacobian matrix, we can test the convexity of the function and calculate local and global optima. 
In this case, the software will update the derivative calculation and output the second derivatives based on the first derivatives(in a 3D tensor if Jacobian is a 2D matrix), 
so there will be challenges associated with implementing tensor operations. 
We will have a new module dedicated to finding the local optima and integrate the output with the GUI.


# Feedback

## Milestone1

#### Feedback from Dovran Amanov: 
On background part: having computational graph example will be nice. - Rest is pretty good


#### Our updates: 
We created the computational graph structure to better describute such calculation in Background/Gaph Structure of Calculation. 
We also updated the directory structure to insert the image we use.


## Milestone 2

Feedback from David Sondag:

Extending AD to higher order derivatives is an excellent idea. To help focus your extension, I recommend that you allow calculation of arbitrary order derivatives for 1D functions (f(x) where f and x are both scalars). Then provide support for the Hessian. Don't worry about supporting higher order objects (greater than order 2) of f(x) for multidimensional x. With these extensions, there is no need to include the extrema calculations, although you are welcome to use these as a demo if you like. Please reach out to myself or Dovran for clarifications.

## Our updates:

- SmartDiff can rapidly and accurate calculate an **arbitrary** order of derivative on real-valued function with univariate input
- SmartDiff also calculates Hessian matrix of real-valued function with multivariate input
In addition to updates for SmartDiff to calculate the Jacobian matrix for vector functions with multivariate input.

Here's how the extension is implemented:

The key modification is on the AD object. It now has a field for the target order of derivative N, and its der[] field is now a size-N vector. The der vector stores all orders of derivative of the AD object up until N. The reason is that based on a formula for calculating nth order derivative of a composite function, we need to use the value of all N derivatives of the inner function. So recursively, AD object requires all N order derivatives in case it is an inner term of some math function later.

Correspondingly, we modified the implementations for each basic operator and math function according to its high-order derivative rule, such that they return an AD object with all orders of derivative calculated.

# Future directions

In the future, we want to make SmartDiff more generalizable and accessible to different users.
First, we plan to extend SmartDiff to calculate Hessian of vector valued functions because it is quite useful in mathematics and application. 
For instance, it can be used to compute higher order Taylor expansion, which might be useful to develp third order or higher order optimization algorithms. Compared with 1st order and 2nd order optimization algorithms, it means we can contain more information and run more accurately within one step. Also, we want SmartDiff to be able to handle vector-valued function that has many vector input with potentially different dimensions. Potentially people will be able to use it more in the future.

Meanwhile, we also hope to optimize our implementation to save running time, so that it can support higher order derivative calculation much better. Right now, we differentiate each component of the function at a time. One thing we can do is to enable multi-parallel computing of different components of the input vector function, since differentiating each function component is independent of one another. Since we are only handling infinitely differentiable functions right now, we can further save computation time by saving intermediate results. For example, in the case of Hessian calculation, the derivative of dxdy is the same as dydx($\frac{\partial ^2f(x,y)}{\partial x\partial y} = \frac{\partial ^2f(x,y)}{\partial y\partial x}$), so we can essentially only calculate dxdy and then use the same value for dydx. On the other hand, we can also extend SmartDiff to handle non-differentiable functions, for example functions with absolute values and sharp corners.

To make SmartDiff more user-friendly, we developed a GUI fronted interface. However, this is just a simplified prototype. In the future, we hope to design it in a more aesthetically pleasing way, and contains more error checks before the user reaches the final step. Also, we can enable the user to change their results in the input checking step. We also aim to improve the documentation of SmartDiff, so it is more accessible to people from a variety of educational background.